In [1]:
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lesechos.json
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json
!pip install feedparser
!pip install newspaper3k

--2023-02-13 14:03:56--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lesechos.json
Resolving people.irisa.fr (people.irisa.fr)... 131.254.254.107
Connecting to people.irisa.fr (people.irisa.fr)|131.254.254.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8276509 (7,9M) [application/json]
Saving to: ‘lesechos.json’

lesechos.json       100%[===================>]   7,89M  11,3MB/s    in 0,7s    

2023-02-13 14:03:57 (11,3 MB/s) - ‘lesechos.json’ saved [8276509/8276509]

--2023-02-13 14:03:57--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json
Resolving people.irisa.fr (people.irisa.fr)... 131.254.254.107
Connecting to people.irisa.fr (people.irisa.fr)|131.254.254.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347 [application/json]
Saving to: ‘sources.json’

sources.json        100%[===================>]     347  --.-KB/s    in 0s      

2023-02-13 14:03:57 (42,9 MB/s) - ‘sources.json’

In [2]:
!python3 -m spacy download fr_core_news_md

2023-02-13 14:05:07.966109: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 14:05:08.265381: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-13 14:05:08.265423: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-13 14:05:09.396963: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
import spacy
nlp = spacy.load('fr_core_news_md')

In [65]:
from urllib.parse import urlparse

import json
f = open('lesechos.json')
data_lesechos = json.load(f)

# (a) se premunir contre le bloquage de commande
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# (b) recuperer le fichier RSS
import feedparser as fp
url = "https://www.lemonde.fr/rss/une.xml"
data = fp.parse(url)

for item in data.entries:
    print(item.title, item.published, item.link)

f = open('sources.json')
liste_rss_flux = json.load(f)
liste_rss_flux = list(liste_rss_flux.values())


liste_new_articles = []
# In lesechos.json, links are the keys but we cannot compare these links to the link we get when we parse RSS links.
# For example, a link from lesechos.json is 
# https://investir.lesechos.fr/marches/actualites/analyse-technique-toute-poursuite-de-la-consolidation-du-contrat-cac-40-sera-limitee-1995443.php?xtor=RSS-24
# While a link from an item entry is 
# https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/la-normalisation-des-ventes-de-velo-apres-le-boom-du-covid-laisse-des-traces-dans-les-comptes-de-thule-1905607
# So we manage to compare only /marches/actualites/analyse-technique-toute-poursuite-de-la-consolidation-du-contrat-cac-40-sera-limitee
liste_url_article_already_in_base = [urlparse(temp).path[:-12] for temp in list(data_lesechos.keys())]
for url in liste_rss_flux:
    data_rss = fp.parse(url)
    for item in data_rss.entries:
        link = item.link
        res = urlparse(link)
        if res.path[:-8] not in liste_url_article_already_in_base:
            liste_new_articles.append(link)
    

Réforme des retraites : les syndicats haussent le ton et prédisent une « journée noire » le 7 mars Mon, 13 Feb 2023 10:25:42 +0100 https://www.lemonde.fr/politique/article/2023/02/13/reforme-des-retraites-les-syndicats-haussent-le-ton-et-predisent-une-journee-noire-le-7-mars_6161625_823448.html
Séisme en Turquie et en Syrie : plus de 35 000 morts et 400 000 personnes évacuées de la région Mon, 13 Feb 2023 14:20:10 +0100 https://www.lemonde.fr/international/article/2023/02/13/seisme-en-turquie-et-en-syrie-le-bilan-toujours-provisoire-s-eleve-a-plus-de-35-000-morts_6161652_3210.html
Alexeï Venediktov : « La haine est entrée au sein de chaque famille russe » Mon, 13 Feb 2023 06:00:49 +0100 https://www.lemonde.fr/international/article/2023/02/13/alexei-venediktov-la-haine-est-entree-au-sein-de-chaque-famille-russe_6161602_3210.html
En Australie, l’alcool de nouveau interdit dans les communautés aborigènes du Territoire du Nord Mon, 13 Feb 2023 03:42:49 +0100 https://www.lemonde.fr/internat

In [73]:
liste_url_article_already_in_base

['/marches/actualites/l-ue-ebauche-un-plan-pour-qualifier-le-gaz-et-le-nucleaire-d-investissements-verts',
 '/marches/actualites/le-gouvernement-veut-eviter-les-penuries-de-main-d-oeuvre-dans-les-entreprises',
 '/marches/analyses-opinions/2022-fera-la-part-belle-au-pricing-power',
 '/marches/analyses-opinions/les-lecons-a-tirer-de-2021',
 '/marches/actualites/wall-street-termine-l-annee-pres-de-ses-records',
 '/marches/actualites/paris-et-new-york-au-zenith',
 '/marches/a-ne-pas-manquer-demain/les-indices-pmi-manufacturiers-pour-commencer-l-annee-boursiere',
 '/marches/actualites/malgre-un-leger-repli-en-fin-de-parcours-le-cac-40-boucle-sa-meilleure-annee-en-plus-de-20-ans',
 '/marches/actualites/enquete-omicron-nouvelle-menace-pour-les-cours-du-petrole',
 '/marches/actualites/l-once-d-or-poursuit-sa-progression-vendredi',
 '/marches/actualites/malgre-son-leger-repli-le-cac-40-est-en-passe-de-boucler-une-annee-faste',
 '/marches/actualites/analyse-technique-le-contrat-sur-le-cac-40-est